In [1]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import json
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
import time
import re

from functions import (get_sector, changes_from_press, get_content_header,
                       get_df, normalise_to_index, save_data)

past_last_date = False

In [4]:
options=Options()

options.set_preference("network.http.pipelining", True)
options.set_preference("network.http.proxy.pipelining", True)
options.set_preference("network.http.pipelining.maxrequests", 8)
options.set_preference("content.notify.interval", 500000)
options.set_preference("content.notify.ontimer", True)
options.set_preference("content.switch.threshold", 250000)
options.set_preference("browser.cache.memory.capacity", 65536) # Increase the cache capacity.
options.set_preference("browser.startup.homepage", "about:blank")
options.set_preference("reader.parse-on-load.enabled", False) # Disable reader, we won't need that.
options.set_preference("browser.pocket.enabled", False) # Duck pocket too!
options.set_preference("loop.enabled", False)
options.set_preference("browser.chrome.toolbar_style", 1) # Text on Toolbar instead of icons
options.set_preference("browser.display.show_image_placeholders", False) # Don't show thumbnails on not loaded images.
options.set_preference("browser.display.use_document_colors", False) # Don't show document colors.
options.set_preference("browser.display.use_document_fonts", 0) # Don't load document fonts.
options.set_preference("browser.display.use_system_colors", True) # Use system colors.
options.set_preference("browser.formfill.enable", False) # Autofill on forms disabled.
options.set_preference("browser.helperApps.deleteTempFileOnExit", True) # Delete temprorary files.
options.set_preference("browser.shell.checkDefaultBrowser", False)
options.set_preference("browser.startup.homepage", "about:blank")
options.set_preference("browser.startup.page", 0) # blank
options.set_preference("browser.tabs.forceHide", True) # Disable tabs, We won't need that.
options.set_preference("browser.urlbar.autoFill", False) # Disable autofill on URL bar.
options.set_preference("browser.urlbar.autocomplete.enabled", False) # Disable autocomplete on URL bar.
options.set_preference("browser.urlbar.showPopup", False) # Disable list of URLs when typing on URL bar.
options.set_preference("browser.urlbar.showSearch", False) # Disable search bar.
options.set_preference("extensions.checkCompatibility", False) # Addon update disabled
options.set_preference("extensions.checkUpdateSecurity", False)
options.set_preference("extensions.update.autoUpdateEnabled", False)
options.set_preference("extensions.update.enabled", False)
options.set_preference("general.startup.browser", False)
options.set_preference("plugin.default_plugin_disabled", False)
options.set_preference("permissions.default.image", 2) # I

driver = webdriver.Firefox(options=options)

driver.get('https://www.jpmorganchase.com/ir/news')

data = []

ticker = 'jpm'

# select by value 

sector = get_sector(ticker)

index_ticker = '^GSPC'
# select by value 

stock_hist = get_df(ticker)
index_hist = get_df(index_ticker)

time.sleep(1)


driver.find_element(By.CLASS_NAME, "view-more-button").click()

html = driver.page_source


soup = BeautifulSoup(html) 

driver.close()

articles = soup.find_all('div',attrs={'class':'item'})

for article in articles:
    
    try:
        date = article.find('p', attrs={'data-dg-test':'date'}).text
    except:
        # jpm has extra rows that contain no data
        break
        
    title = article.find('p', attrs={'class':'dg-role press-release'}).text.lstrip().rstrip() 
    
    date = date.lstrip().rstrip()    # remove starting and trailing whitespaces
    title = re.sub("\s\s+", " ", title)

    date = datetime.strptime(date, '%B %d, %Y').date()
    
    url = article.find('a').get('href')
    url = f'https://www.jpmorganchase.com{url}'
    print(url)
    
    if date < datetime(2014, 1, 1).date():
        past_last_date = True
        break
    
    pct_change = changes_from_press(stock_hist, date, 1)

    if pct_change != None:
        index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
    else:
        index_price, norm_price = (None, None)

    data.append([date, title, url, pct_change, index_price, norm_price])


save_data(data, columns, ticker, sector)

https://www.jpmorganchase.com/ir/news/2023/jpmc-declares-common-stock-dividend-5-15
https://www.jpmorganchase.com/ir/news/2023/jpmc-to-present-at-morgan-stanley-us-financials-payments-cre-conference
https://www.jpmorganchase.com/ir/news/2023/jpmorgan-chase-2023-annual-meeting-of-shareholders
https://www.jpmorganchase.com/ir/news/2023/jpmc-investor-day-2023
https://www.jpmorganchase.com/ir/news/2023/jpmorgan-chase-announces-conference-calls-to-review-first-quarter-second-quarter-third-quarter-and-fourth-quarter-2024-earnings
https://www.jpmorganchase.com/ir/news/2023/jpmc-to-present-at-alliance-bernstein-strategic-decisions-conference
https://www.jpmorganchase.com/ir/news/2023/jpmc-acquires-substantial-majority-of-assets-and-assumes-certain-liabilities-of-first-republic-bank
https://www.jpmorganchase.com/ir/news/2023/jpmc-announces-replacement-rate-for-outstanding-securities-and-other-instruments-linked-to-usd-libor-isr
https://www.jpmorganchase.com/ir/news/2023/jpmc-declares-preferred-

https://www.jpmorganchase.com/ir/news/2020/jpmc-names-reggie-chambers-the-new-head-of-investor-relations
https://www.jpmorganchase.com/ir/news/2020/JPMorgan-Chase-Declares-Preferred-Stock-Dividends-11-13-2020
https://www.jpmorganchase.com/ir/news/2020/jpmc-to-present-at-the-goldman-sachs-us-financial-services-conference
https://www.jpmorganchase.com/ir/news/2020/jpmc-elects-phebe-novakovic-to-its-board-of-directors
https://www.jpmorganchase.com/ir/news/2020/JPMorgan-Chase-Declares-Preferred-Stock-Dividends-10-15-2020
https://www.jpmorganchase.com/ir/news/2020/jpmc-commits-30-billion-to-advance-racial-equity
https://www.jpmorganchase.com/ir/news/2020/adopts-paris-aligned-financing-commitment
https://www.jpmorganchase.com/ir/news/2020/jpmc-host-3rd-qtr-earnings-call-100620
https://www.jpmorganchase.com/ir/news/2020/jpmorgan-chase-to-present-at-the-bofa-10-05-2020
https://www.jpmorganchase.com/ir/news/2020/jpmorgan-chase-to-present-at-the-bancanalysts-association-of-bos
https://www.jpmorg

https://www.jpmorganchase.com/ir/news/2017/common-stock-dividend-091917
https://www.jpmorganchase.com/ir/news/2017/preferred-stock-dividend-091517
https://www.jpmorganchase.com/ir/news/2017/preferred-stock-dividend-081517
https://www.jpmorganchase.com/ir/news/2017/investor-day-2018
https://www.jpmorganchase.com/ir/news/2017/barclays-global-financial-services-conf-72117
https://www.jpmorganchase.com/ir/news/2017/preferred-stock-dividend-071417
https://www.jpmorganchase.com/ir/news/2017/warrant-exercise-price-070317
https://www.jpmorganchase.com/ir/news/2017/2017-capital-distribution-plan
https://www.jpmorganchase.com/ir/news/2017/2017-dfast
https://www.jpmorganchase.com/ir/news/2017/preferred-stock-dividend-061517
https://www.jpmorganchase.com/ir/news/2017/2q17-earnings
https://www.jpmorganchase.com/ir/news/2017/common-stock-dividend-051617
https://www.jpmorganchase.com/ir/news/2017/preferred-stock-dividend-051517
https://www.jpmorganchase.com/ir/news/2017/2017-annual-meeting-shareholde

https://www.jpmorganchase.com/ir/news/2014/id-857440
https://www.jpmorganchase.com/ir/news/2014/id-854256
https://www.jpmorganchase.com/ir/news/2014/id-853140
https://www.jpmorganchase.com/ir/news/2014/id-851213
https://www.jpmorganchase.com/ir/news/2014/id-850162
https://www.jpmorganchase.com/ir/news/2014/id-849126
https://www.jpmorganchase.com/ir/news/2014/id-848529
https://www.jpmorganchase.com/ir/news/2014/id-847590
https://www.jpmorganchase.com/ir/news/2014/id-847589
https://www.jpmorganchase.com/ir/news/2014/id-846974
https://www.jpmorganchase.com/ir/news/2014/id-844134
https://www.jpmorganchase.com/ir/news/2014/id-840340
https://www.jpmorganchase.com/ir/news/2014/id-838083
https://www.jpmorganchase.com/ir/news/2014/id-835857
https://www.jpmorganchase.com/ir/news/2014/id-835247
https://www.jpmorganchase.com/ir/news/2014/id-834318
https://www.jpmorganchase.com/ir/news/2014/id-833987
https://www.jpmorganchase.com/ir/news/2014/id-833630
https://www.jpmorganchase.com/ir/news/2014/id-